# 목차

* 학습 목표
* 데이터 준비와 전처리
* 모델 학습하기
* 영화 추천 받기
* 회고

# 학습 목표 

루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

----------

평가문항	상세기준

1. CSR matrix가 정상적으로 만들어졌다.

> 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.

> 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

> MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.

In [20]:
!pip install --upgrade pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install implicit==0.4.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.6
1.7.3
0.4.8


In [23]:
import pandas as pd


# 데이터 준비와 전처리

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 데이터 준비

In [52]:
rating_file_path='/content/drive/MyDrive/아이펠 데이터/exp13_추천시스템/data/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [53]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [54]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [55]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [56]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'counts']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [57]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='/content/drive/MyDrive/아이펠 데이터/exp13_추천시스템/data/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [131]:
print(movies[movies['movie_id']==1]['title'])

0    Toy Story (1995)
Name: title, dtype: object


## 데이터 분석

In [59]:
# 영화 수
ratings['movie_id'].nunique()

3628

In [60]:
# 유저 수
ratings['user_id'].nunique()

6039

In [61]:
# 인기 많은 영화 30개
artist_count = ratings.groupby('movie_id')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [62]:
# 유저별 몇 개의 영화를 보았는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [63]:
ratings.describe()

,user_id,movie_id,counts
count,836478.000000,836478.000000,836478.000000
mean,3033.120626,1849.099114,3.958293
std,1729.255651,1091.870094,0.762280
min,1.000000,1.000000,3.000000
25%,1531.000000,1029.000000,3.000000
50%,3080.000000,1747.000000,4.000000
75%,4485.000000,2763.000000,5.000000
max,6040.000000,3952.000000,5.000000


In [64]:
# 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 주자. 
my_favorite = [1 , 29 , 2858 , 260 , 1196]

# '3953'이라는 user_id가 위 영화를 5회씩 보았다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'counts':5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 '6041'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,1,5
1,6041,29,5
2,6041,2858,5
3,6041,260,5
4,6041,1196,5


## CSR(Compressed Sparse Row) Matrix

In [89]:
# CSR 행렬
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings.user_id.max()+1
num_movie = ratings.movie_id.max()+1

csr_ratings = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_ratings

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 모델 학습하기

In [113]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

처음에는 factors=100, iterations=15 로 진행하였으나 모델 성능을 향상시키기 위해 수치를 늘려서 다시 학습했다.

In [114]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=25, dtype=np.float32)

In [115]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [116]:
# 모델 훈련
als_model.fit(csr_ratings_transpose)

  0%|          | 0/25 [00:00<?, ?it/s]

In [117]:
np.shape(als_model.user_factors)

(6042, 200)

In [118]:
me_vector, Toy_Story_vector, Jumanji_vector = als_model.user_factors[6041], als_model.item_factors[1], als_model.item_factors[2]

print(me_vector)

[-3.01237196e-01 -7.15002343e-02 -2.66290903e-01  8.08786213e-01
  2.85090506e-01  4.79326427e-01 -1.22726142e-01 -2.76526183e-01
 -2.44383097e-01  2.41598129e-01 -4.17976946e-01 -3.90627801e-01
 -2.77597517e-01 -3.75827402e-01 -1.64066195e-01 -3.31705064e-02
  5.38211763e-01  2.78370380e-01 -1.43366084e-02 -3.83500189e-01
  2.99701989e-01 -5.57929389e-02  9.65293646e-02  1.70125410e-01
  2.57658154e-01  3.15725654e-01 -3.99420738e-01  1.08310647e-01
  2.55518943e-01  3.32848839e-02  2.28340477e-01  2.95595706e-01
 -3.71129662e-02 -1.15689270e-01  8.33958507e-01 -4.29112434e-01
 -4.36374605e-01 -3.64542544e-01  5.07169724e-01 -4.29887101e-02
  3.44085872e-01 -5.64766943e-01 -3.18187028e-01  2.52248980e-02
 -3.23109701e-02  4.44234014e-01 -7.50042498e-01 -1.42542914e-01
  7.30861127e-02  3.55151594e-01 -3.03183436e-01 -4.07928914e-01
 -1.29550353e-01  2.08062738e-01 -1.82580855e-02 -6.99118897e-02
  5.81458688e-01 -6.76179707e-01 -4.59553227e-02 -2.54978299e-01
  5.01303256e-01  1.66720

In [119]:
Toy_Story_vector

array([ 1.13161830e-02, -2.64830235e-03, -6.56355824e-03,  3.47598060e-03,
        6.77310675e-03,  1.46951713e-02, -1.81434676e-02, -9.64516215e-03,
       -2.98103336e-02, -4.61298181e-03,  2.89685628e-03, -1.67800970e-02,
       -2.88606025e-02, -8.16168729e-03,  1.07474895e-02, -9.80384182e-04,
        3.27427834e-02, -3.41008906e-03, -1.62979160e-02,  7.43287243e-03,
        3.95099679e-03, -1.52572133e-02, -2.47359313e-02,  1.22764260e-02,
        1.59497131e-02, -5.37877809e-03,  1.59154665e-02,  1.80993155e-02,
       -1.43961376e-02,  6.69703772e-03,  1.03691742e-02,  1.18238516e-02,
        1.91484615e-02,  1.74280535e-02,  5.23734745e-03,  1.78560670e-02,
       -1.68758035e-02, -7.99811538e-03,  2.83698998e-02,  9.33888368e-03,
        2.03074943e-02,  2.17489130e-03,  2.15315912e-03,  2.22330429e-02,
        2.09549312e-02, -1.02731381e-02,  3.08360416e-03,  1.04982750e-02,
       -9.12570395e-03,  1.26597863e-02, -5.06536011e-03,  9.32403188e-03,
       -1.38402190e-02, -

In [120]:
Jumanji_vector

array([-1.32106859e-02,  3.87104265e-02,  9.45161097e-04,  1.12666441e-02,
       -1.34573309e-02, -1.38417343e-02, -2.99877711e-02,  3.53351794e-03,
       -5.25662815e-03, -1.82427708e-02,  4.46447078e-03, -1.10082161e-02,
        1.61263708e-03,  1.38277402e-02,  1.28648672e-02,  4.49282117e-03,
        1.29679330e-02,  6.33034343e-03,  2.01682691e-02, -1.72477867e-03,
       -1.42511132e-03,  9.07743908e-03, -4.28356091e-03,  2.88057351e-03,
       -1.80183200e-03, -6.33623917e-03, -3.99884069e-03, -1.71985209e-03,
        1.93481278e-02,  8.60074442e-03,  1.14014568e-02,  1.40743107e-02,
        4.45030928e-02,  9.24917031e-03,  1.91425141e-02, -1.65594593e-02,
        7.85261672e-03, -1.52380196e-02,  1.62897687e-02,  1.28478827e-02,
        1.20730000e-02,  1.72082335e-02,  3.37266611e-05, -9.99909546e-03,
        1.11201163e-02,  1.34869246e-02,  5.66419214e-03, -3.02443723e-03,
       -2.03482248e-03, -2.38597598e-02, -1.67858191e-02, -1.05294669e-02,
        2.43482478e-02,  

In [121]:
# me와 Toy_Story를 내적하는 코드
np.dot(me_vector, Toy_Story_vector)

0.64274406

In [122]:
# me와 Jumanji를 내적하는 코드
np.dot(me_vector, Jumanji_vector)

0.04388515

# 영화 추천 받기

## 비슷한 영화 찾기

In [132]:
# 토이 스토리와 비슷한 영화 찾기
similar_movie = als_model.similar_items(1, N=15)
similar_movie

[(1, 1.0000001),
 (3114, 0.54577535),
 (588, 0.35402784),
 (2355, 0.3466607),
 (34, 0.32231),
 (1265, 0.3046557),
 (364, 0.30336314),
 (106, 0.2785636),
 (595, 0.2753537),
 (1566, 0.26169538),
 (1920, 0.2585002),
 (1473, 0.25755847),
 (559, 0.2567972),
 (3796, 0.2509375),
 (854, 0.25038612)]

In [134]:
print(movies[movies['movie_id']==3114]['title'])

Series([], Name: title, dtype: object)


In [135]:
print(movies[movies['movie_id']==588]['title'])

584    Aladdin (1992)
Name: title, dtype: object


자기 자신을 제외하고 Toy Story 와 가장 비슷한 영화를 찾아본 결과, 상위 2개를 보니 Toy Story 2 와 Aladdin 이 추천되었음을 확인할 수 있었다. 

## 영화 추천 받기

AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통해 좋아할 만한 아티스트를 추천받는다. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument이다.

In [136]:
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(6041, csr_ratings, N=20, filter_already_liked_items=True)
artist_recommended

[(1210, 0.6959924),
 (3114, 0.39736062),
 (1198, 0.34678498),
 (1968, 0.25392047),
 (2571, 0.24661247),
 (2997, 0.21795872),
 (2396, 0.20210692),
 (2628, 0.19776239),
 (924, 0.19395953),
 (2355, 0.17408553),
 (1214, 0.16718647),
 (750, 0.16377306),
 (1270, 0.16094126),
 (588, 0.1594357),
 (2600, 0.15768252),
 (1923, 0.15037411),
 (1721, 0.14984623),
 (858, 0.13915017),
 (1175, 0.1367032),
 (3000, 0.13527879)]

In [137]:
print(movies[movies['movie_id']==1210]['title'])

1192    Star Wars: Episode VI - Return of the Jedi (1983)
Name: title, dtype: object


In [138]:
print(movies[movies['movie_id']==1198]['title'])

1180    Raiders of the Lost Ark (1981)
Name: title, dtype: object


상위 3개의 영화를 보니 스타 워즈 6, 토이스토리 2, 레이더스가 추천되었다.

이 추천에 데이터가 얼마나 기여했는지 확인해보자

In [140]:
explain = als_model.explain(6041, csr_ratings, itemid=1210)

In [142]:
explain[1]

[(1196, 0.515412645394323),
 (29, 0.07178617587031033),
 (260, 0.07054603322703788),
 (2858, 0.04289926063692712),
 (1, -0.014490336717289084)]

In [143]:
print(movies[movies['movie_id']==1196]['title'])

1178    Star Wars: Episode V - The Empire Strikes Back...
Name: title, dtype: object


스타 워즈 6편을 추천하는데에 스타 워즈 5편이 가장 큰 기여를 했음을 알 수 있다.

실제로 같은 시리즈로 서로 연관이 매우 깊으므로 영화끼리의 관계를 잘 파악하고 있다고 할 수 있다.

# 회고

이번 노드에서는 특정한 개인에게 영화를 추천해주는 모델을 만들었다. 사용자가 좋아하는 영화 데이터가 있으면 그것들과 유사한 영화를 추천해주는 것이다. 하지만 영화들에 대한 데이터는 없고, 오직 어떤 사람들이 어떤 영화를 보고 어떤 평점을 매겼는가에 대한 데이터셋만을 가지고 진행한다는 점에서 기존의 노드와는 달랐다.

나는 수많은 사람들이 평점을 매긴 데이터만을 가지고 특정 영화 5개를 선호하는 유저를 만들어 추가했다. 각각의 유저들은 자신만의 취향이 있으며 이로 인해 서로 유사한 영화들을 선호하리라 예상할 수 있다. 따라서 그들의 경향성을 살펴보면 우리가 비록 영화들에 대한 데이터가 따로 없더라도 영화들끼리의 관계를 파악할 수 있게 될 것이다.

실제로 MF 모델을 통해 서로 비슷한 영화들을 찾아내는 데에 성공했다. 사람들의 평점만을 가지고 가장 관련있는 영화로 같은 시리즈 내의 다른 후속작들을 찾아낸 것이다. 그 외에도 장르나 분위기가 비슷한 영화들도 잘 찾아내었고 이를 통해 영화 추천도 성공적으로 해냈다.

하지만 중간에 이해하기 힘든 부분이 있었다. 바로 모델 학습 후 벡터로 나타내는 파트였다. 기존에 미리 좋아하는 영화로 집어넣은 영화를 다시 그 유저와 내적했을 때 나는 1 혹은 1에 가까운 값이 나올 줄 알았으나 처음에 0.46 이라는 낮은 값이 나왔다. 모델 학습이 덜 된 것으로 판단하고 학습을 더 진행한 후에도 0.64 정도의 수치만 나왔을 뿐이다. 그러나 노드를 좀 더 진행한 후에 어째서 그랬는지 알 수 있었다. 영화 추천은 내적값이 가장 높은 순서대로 나오므로 절대적인 값은 그리 중요하지 않았던 것이다. 0.3 정도만 되어도 아주 높은 추천 순위에 있으며 결국 중요한 것은 어떤 수치 이상일 때 추천할만한가를 설정하는 부분에 있었다는 사실을 깨달았다.

마지막으로 아쉬운 점이 있었다면 쥬만지의 내적 값이 아주 작게 나왔다는 것이다. 영화 추천을 통해 분명 유사한 영화를 찾아낼 수 있었지만 그럼에도 일부 영화는 비슷함에도 불구하고 찾아내지 못 했다는 점을 알 수 있었다.

하지만 영화 추천에서는 일부 추천할만한 영화를 찾아내지 못 하는 것보다, 전혀 생뚱맞은 영화를 추천해버리는 것이 더 나쁜 일이므로 일어날 수 있는 경우라고 생각한다.